# HW3

## Text Processing

### Q1

1. Modify the code I wrote in lecture 8 with what you have learnt in lecture 9 and correctly tokenize the text both on the word and sentence level, and by removing the stopwords. Rewrite the `getSummary` function and all the other functions that it depends by maing these corrections.

2. Rewrite the code I wrote for `getKeywords` function making the same corrections.

3. Test your code from parts 1 and 2 on random articles from the Guardian.

4. Rewrite the `getSubjectGuardian` function for another newspaper in English, and test your code from part 1 and 2 on random articles from this new newspaper.

In [1]:
import nltk
import numpy as np
import pandas as pd
import requests
import regex as re
import spacy

from nltk.corpus import stopwords
from xmltodict import parse
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
def getMatrix(sentences,sw):
    vectorizer = CountVectorizer(stop_words=sw)
    return vectorizer.fit_transform(sentences)

In [3]:
def getSummary(text,k,sw):
    sentences = sent_tokenize(text)
    sw= set(stopwords.words('english'))
    matrix = getMatrix(sentences,sw)
    projection = PCA(n_components=1)
    weights = projection.fit_transform(matrix.toarray())
    res = list(zip(weights.transpose()[0],range(112),sentences))
    tmp = sorted(res,key=lambda x: x[0],reverse=True)[:k]
    return sorted(tmp, key=lambda x: x[1])

In [4]:
def getKeywords(text,sw,k):
    sentences = sent_tokenize(text)
    
    vectorizer = CountVectorizer(stop_words=sw)
    matrix = vectorizer.fit_transform(sentences)
    words = vectorizer.get_feature_names()
    
    projection = PCA(n_components=1)
    tmp = projection.fit_transform(matrix.transpose().toarray())
    weights = tmp.transpose()[0]
    
    return sorted(zip(weights,words),key=lambda x: x[0], reverse=True)[:k]

In [5]:
def get_text(url):
    with requests.get(url) as link:
        r = BeautifulSoup(link.content,'html.parser')
    text=' '.join([x.text for x in r.find_all('p')])
    return text

In [6]:
def random_theguardian_url(section):
    with requests.get(f'https://www.theguardian.com/{section}/rss') as link:
        raw = parse(link.text)
    n=len(raw['rss']['channel']['item'])
    m = np.random.randint(n)
    return raw['rss']['channel']['item'][m]['link']

In [7]:
text=get_text(random_theguardian_url('sport'))
text

'Joel Embiid had 33 points and 10 rebounds, James Harden added 22 points and 15 assists and the Philadelphia 76ers beat the Toronto Raptors 132-97 in Game 6 to end the first-round series Thursday night. Philadelphia will face Miami in the second round. The top-seeded Heat beat Atlanta in five games. Tyrese Maxey scored 25 points, going 5 for 12 from three-point range, and Tobias Harris had 19 points and 11 rebounds, to help the 76ers finish off the Raptors with a blowout after losing the past two games. Danny Green scored 12 points and shot 4 for 7 from three-point range as the Sixers finished 16 for 40 from beyond the arc. The Raptors struggled from beyond the arc, going 7 for 35. Chris Boucher had 25 points and 10 rebounds for Toronto, Pascal Siakam fouled out with 24 points, Gary Trent Jr had 19 and Scottie Barnes 18. Toronto lost in the first round for the first time since being swept by Washington in 2015. Toronto played without all-star guard Fred VanVleet. He missed his second s

In [8]:
sw = set(stopwords.words('english'))
getSummary(text,5,sw)

[(2.1477626958340594,
  0,
  'Joel Embiid had 33 points and 10 rebounds, James Harden added 22 points and 15 assists and the Philadelphia 76ers beat the Toronto Raptors 132-97 in Game 6 to end the first-round series Thursday night.'),
 (4.094981710624469,
  16,
  'Chris Paul went 14 for 14 in the best shooting night in NBA playoff history, Devin Booker returned from a hamstring injury to hit a late three-pointer and the Phoenix Suns beat the New Orleans Pelicans 115-109 on Thursday night to finish off the first-round series in six games.'),
 (1.3020384457535252,
  24,
  'Booker played for the first time in nine days since injuring his right hamstring after scoring 31 points in the first half of Game 2.'),
 (1.68945894127176,
  28,
  'Luka Doncic and Jalen Brunson each scored 24 points and the Dallas Mavericks beat the Utah Jazz 98-96 on Thursday night in Game 6 to wrap up the first-round series.'),
 (1.2920967299404738,
  34,
  'The Mavericks won in the first round for the first time s

In [9]:
getKeywords(text,sw,5)

[(4.573113121770563, 'points'),
 (1.5818799257271328, 'rebounds'),
 (1.4509763265918432, 'first'),
 (1.163117951293904, '10'),
 (1.159907488762304, '19')]

In [10]:
def getSubject_independent(subject):
    with requests.get(f'https://www.independent.co.uk/{subject}/rss') as link:
        raw = parse(link.text)
    n=len(raw['rss']['channel']['item'])
    m = np.random.randint(n)
    return raw['rss']['channel']['item'][m]['link']

In [11]:
text=get_text(getSubject_independent('/news/uk/politics'))[137:1956]
text

'Minister admits controls would have hit supermarkets prices and been ‘act of self-harm’  Find your bookmarks in your Independent Premium section, under my profile Boris Johnson’s government has scrapped the introduction of planned post-Brexit inspections on food coming into the UK from the EU, cabinet minister Jacob Rees-Mogg has announced. In a written statement to the Commons, the Brexit opportunities minister said the controls set to come into force on 1 July would not be enforced during 2022. “No further import controls on EU goods will be introduced this year – businesses can stop their preparations for July now,” Mr Rees-Mogg said. Mr Rees-Mogg suggested the government planned a complete overhaul of its border check plans, revealing that the end of 2023 was the target for a brand new “controls regime”. The Independent revealed last month that the government was set to push back the import controls because of growing fears they will exacerbate the\xa0cost of living crisis. Mr Ree

In [12]:
getSummary(text,5,sw)

[(4.8912976942003885,
  0,
  'Minister admits controls would have hit supermarkets prices and been ‘act of self-harm’  Find your bookmarks in your Independent Premium section, under my profile Boris Johnson’s government has scrapped the introduction of planned post-Brexit inspections on food coming into the UK from the EU, cabinet minister Jacob Rees-Mogg has announced.'),
 (-0.13150562451221526,
  1,
  'In a written statement to the Commons, the Brexit opportunities minister said the controls set to come into force on 1 July would not be enforced during 2022.'),
 (1.435645702939244,
  6,
  'However, he later told ITV that the controls on EU imports “would have been an act of self-harm” – admitting that post-Brexit checks would have seen prices rise at the supermarket.'),
 (-0.4181685099653282,
  8,
  '“Free trade is hugely advantageous to consumers.” The Sanitary and Phytosanitary (SPS)\xa0checks planned for on 1 July would have seen checks on imported agri-food and plant imports.'),


In [13]:
getKeywords(text,sw,5)

[(1.584214879640418, 'minister'),
 (1.3545126801294898, 'would'),
 (1.1181374058047742, 'brexit'),
 (1.0723871521390806, 'controls'),
 (1.0327538333256236, 'eu')]

### Q2

Write a function that returns all named entities (proper names, country names, corporation names only) from a URL. Function should take the URL as the input and must return the list of named entities from that URL. Test your code on random articles from the Guardian. Don't use the NLTK's NER that I demonstrated during the lecture. Use the SpaCY's NER function.

In [14]:
article_link=random_theguardian_url('sport')

In [15]:
def named_entities(url):
    text = get_text(url)
    NER = spacy.load("en_core_web_sm")
    doc = NER(text)

    for ent in doc.ents:
        if ((ent.label_=='PERSON') or (ent.label_=='ORG') or (ent.label_=='GPE')):
            print(ent.text,ent.label_)

In [16]:
named_entities(article_link)

Red Roses ORG
England GPE
Simon Middleton PERSON
France GPE
England GPE
France GPE
New Zealand GPE
England GPE
Eddie Jones PERSON
the Six Nations ORG
Middleton PERSON
England GPE
France GPE
England PERSON
Emily Scarratt PERSON
the Six Nations ORG
England GPE
France GPE
England GPE
Bayonne ORG
BBC2 ORG
England GPE
France GPE
England GPE
Antoine Dupont’s ORG
France GPE
Scotland GPE
Ireland GPE
Wales GPE
England GPE
England GPE
England GPE
Bayonne knocks PERSON
England GPE
Middleton ORG
the Black Ferns ORG
England GPE
France GPE
Wayne Smith PERSON
England GPE
France GPE
Middleton PERSON
Middleton ORG
the British & Irish Lions ORG
Super Rugby ORG
Ian Foster PERSON
Springboks ORG
the Six Nations ORG
England GPE
France GPE
Middleton PERSON


### Q3

1. Write a function that returns the most positive and the most negative sentences from a text. The function must take the text as the input and must return a 2-tuple: the first element as the most positive and the second as the most negative sentence with their polarity scores.

2. Test your function on random articles from the Guardian.

In [18]:
article_link=random_theguardian_url('sport')

In [20]:
text=get_text(article_link)
text

'Make or Break’s behind-the-scenes look at the World Surf League engages fans and non-fans alike even if some hard questions remain ignored There is a moment early in the first episode of Make or Break, a new series from Apple TV+ about the World Surf League, when Tyler Wright neatly encapsulates the tension at the heart of professional surfing. “We come from a sport of, ‘aww we’re hippies’,” says the Australian surfer, a two-time WSL champion. “We’re not. We’re competitive little assholes.” Surfing has long grappled with this bipolar identity; a sport initially beloved by the counter-culture movement that grew popular, some would say too popular, on the back of booming commercial and competitive success. Make or Break, an entertaining seven-part series, is the latest attempt by the surfing elite to capitalise on its popularity. Given the show has already been renewed for a second season – before it even went to air – they will most likely succeed. It seems that no high-performance spo

In [21]:
def analyze(text):
    
    analyzer = SentimentIntensityAnalyzer()
    
    analyzer.polarity_scores(text)
    sentences = sent_tokenize(text)
    sentences=[(x,analyzer.polarity_scores(x)) for x in sentences]

    max_pol=sentences[0][1]['compound']
    min_pol=sentences[0][1]['compound']
    max_sent=sentences[0][0]
    min_sent=sentences[0][0]
    for i in sentences:
        if i[1]['compound']>max_pol:
            max_pol=i[1]['compound']
            max_sent=i[0]
        elif i[1]['compound']<min_pol:
            min_pol=i[1]['compound']
            min_sent=i[0]
    max_list=[max_sent,max_pol]
    min_list=[min_sent,min_pol]
    return max_list,min_list

In [22]:
analyze(text)

(['We’re competitive little assholes.” Surfing has long grappled with this bipolar identity; a sport initially beloved by the counter-culture movement that grew popular, some would say too popular, on the back of booming commercial and competitive success.',
  0.9325],
 ['But a shark attack at Maui during last season’s event saw the women forced to finish the event at Pipeline, a famously-heavy wave.',
  -0.8462])